# Simularium Conversion Tutorial : Custom Data

In [1]:
from IPython.display import Image
import numpy as np
from simulariumio import (
    TrajectoryConverter, 
    TrajectoryData, 
    AgentData, 
    UnitData, 
    MetaData, 
    ModelMetaData,
    CameraData, 
    DisplayData,
    DISPLAY_TYPE,
)

This notebook provides example python code for converting your own simulation trajectories into the format consumed by the Simularium Viewer. It creates a .simularium file which you can drag and drop onto the viewer like this:

![title](img/drag_drop.gif)

***
## Prepare your spatial data

The Simularium custom data Converter consumes spatiotemporal data from any source. It requires a `TrajectoryData` object as parameter ([see documentation](https://simularium.github.io/simulariumio/simulariumio.data_objects.html#simulariumio.data_objects.trajectory_data.TrajectoryData)).

If you'd like to specify PDB or OBJ files or color for rendering an agent type, add a `DisplayData` object for that agent type, as shown below ([see documentation](https://simularium.github.io/simulariumio/simulariumio.data_objects.html#module-simulariumio.data_objects.display_data)).


### Generate example data

To demonstrate using the custom converter, we'll first generate some random example data.

#### If there are NO fibers in the data:

In [2]:
from string import ascii_uppercase
from random import choice

# parameters
total_steps = 5
timestep = 0.5
box_size = 100
n_agents = 5
min_radius = 5
max_radius = 10

type_names = []
for t in range(total_steps):
    type_names.append([choice(ascii_uppercase) for i in range(n_agents)])
    
example_default_data = TrajectoryData(
    meta_data=MetaData(
        box_size=np.array([box_size, box_size, box_size]),
        camera_defaults=CameraData(
            position=np.array([10.0, 0.0, 200.0]),
            look_at_position=np.array([10.0, 0.0, 0.0]),
            fov_degrees=60.0,
        ),
        trajectory_title="Some parameter set",
        model_meta_data=ModelMetaData(
            title="Some agent-based model",
            version="8.1",
            authors="A Modeler",
            description=(
                "An agent-based model run with some parameter set"
            ),
            doi="10.1016/j.bpj.2016.02.002",
            source_code_url="https://github.com/simularium/simulariumio",
            source_code_license_url="https://github.com/simularium/simulariumio/blob/main/LICENSE",
            input_data_url="https://allencell.org/path/to/native/engine/input/files",
            raw_output_data_url="https://allencell.org/path/to/native/engine/output/files",
        ),
    ),
    agent_data=AgentData(
        times=timestep * np.array(list(range(total_steps))),
        n_agents=np.array(total_steps * [n_agents]),
        viz_types=np.array(total_steps * [n_agents * [1000.0]]),  # default viz type = 1000
        unique_ids=np.array(total_steps * [list(range(n_agents))]),
        types=type_names,
        positions=np.random.uniform(size=(total_steps, n_agents, 3)) * box_size - box_size * 0.5,
        radii=(max_radius - min_radius) * np.random.uniform(size=(total_steps, n_agents)) + min_radius,
        display_data={
            type_names[0][0]: DisplayData(
                name="Molecule A",
                display_type=DISPLAY_TYPE.PDB,
                url="https://files.rcsb.org/download/3KIN.pdb",
                color="#0080ff",
            ),
        },
    ),
    time_units=UnitData("ns"),  # nanoseconds
    spatial_units=UnitData("nm"),  # nanometers
)

#### Otherwise:

In [3]:
from string import ascii_uppercase
from random import choice

# parameters
total_steps = 5
timestep = 0.5
box_size = 100
n_agents = 5
min_radius = 5
max_radius = 10
points_per_fiber = 4

type_names = []
for t in range(total_steps):
    type_names.append([choice(ascii_uppercase) for i in range(n_agents)])

example_fiber_data = TrajectoryData(
    meta_data=MetaData(
        box_size=np.array([box_size, box_size, box_size]),
        camera_defaults=CameraData(
            position=np.array([10.0, 0.0, 200.0]),
            look_at_position=np.array([10.0, 0.0, 0.0]),
            fov_degrees=60.0,
        ),
        trajectory_title="Some parameter set",
        model_meta_data=ModelMetaData(
            title="Some agent-based model",
            version="8.1",
            authors="A Modeler",
            description=(
                "An agent-based model run with some parameter set"
            ),
            doi="10.1016/j.bpj.2016.02.002",
            source_code_url="https://github.com/simularium/simulariumio",
            source_code_license_url="https://github.com/simularium/simulariumio/blob/main/LICENSE",
            input_data_url="https://allencell.org/path/to/native/engine/input/files",
            raw_output_data_url="https://allencell.org/path/to/native/engine/output/files",
        ),
    ),
    agent_data=AgentData(
        times=timestep * np.array(list(range(total_steps))),
        n_agents=np.array(total_steps * [n_agents]),
        viz_types=np.array(total_steps * [n_agents * [1001.0]]), # fiber viz type = 1001
        unique_ids=np.array(total_steps * [list(range(n_agents))]),
        types=type_names,
        positions=np.zeros(shape=(total_steps, n_agents, 3)),
        radii=np.ones(shape=(total_steps, n_agents)),
        n_subpoints=points_per_fiber * np.ones(shape=(total_steps, n_agents)),
        subpoints=box_size * np.random.uniform(
            size=(total_steps, n_agents, points_per_fiber * 3)) - box_size * 0.5,
        display_data={
            type_names[0][0]: DisplayData(
                name="X",
                color="#0080ff",
                display_type=DISPLAY_TYPE.SPHERE,
            ),
        },
    ),
    time_units=UnitData("ns"),  # nanoseconds
    spatial_units=UnitData("nm"),  # nanometers
)

## Convert and save as .simularium file

Once your data is shaped like in the `example_default_data` or `example_fiber_data` object, you can use the converter to generate the file at the given path:

In [4]:
TrajectoryConverter(example_fiber_data).save("example_fibers")
TrajectoryConverter(example_default_data).save("example_default")

Converting Trajectory Data to Binary -------------
Writing Binary -------------
saved to example_fibers.simularium
Converting Trajectory Data to Binary -------------
Writing Binary -------------
saved to example_default.simularium


## Visualize in the Simularium viewer

In a supported web-browser (Firefox or Chrome), navigate to https://simularium.allencell.org/ and import your file into the view.